#### 4-10. 분산의 중요성
* 평균값은 동일하지만 분산값이 다름으로 인해 전체적인 데이터의 모습이 완전히 달라 보임
* 분산이 클수록 집단의 평균값의 차이가 무의미해진다.
* **집단간 평균값의 분산이 클수록, 집단 내 분산이 작아질 수록 평균의 차이가 분명해진다.**

#### 4-02. 일원분산분석(One-way ANOVA)
* 독립변수: 범주형 1개, 종속변수: 연속형
* 독립변수의 변화가 종속변수에 미치는 영향을 보기 위해 사용
* 가정: 독립성, 정규성, 등분산성
* scipy.stats나 statsmodels 라이브러리 사용

* **iris 데이터에서, 품종(species)별 sepal_length의 평균에 차이가 있을까?**
* 독립변수: species
* 종속변수: sepal_length, sepal_width, petal_length, petal_width 중 1개
* One-way ANOVA를 통해 평균 차이에 대한 통계적 유의성을 알아 볼 수 있다.
* **귀무가설: 세 품종의 sepal_width 평균은 동일하다.**
* **대립가설: 적어도 한 품종의 sepal_width 평균이 다르다.**

In [4]:
# [1] 데이터 가져오기
import pandas as pd
import seaborn as sns

iris = sns.load_dataset('iris')
print(iris.head(3))

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa


In [8]:
# [2] 그룹나누기
target = 'sepal_width'
# print(iris['species'].unique())
gA, gB, gC = [s.to_numpy() for _, s in iris.groupby('species')[target]]
print(gA, gB, gC, sep="\n")

[3.5 3.  3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 3.7 3.4 3.  3.  4.  4.4 3.9 3.5
 3.8 3.8 3.4 3.7 3.6 3.3 3.4 3.  3.4 3.5 3.4 3.2 3.1 3.4 4.1 4.2 3.1 3.2
 3.5 3.6 3.  3.4 3.5 2.3 3.2 3.5 3.8 3.  3.8 3.2 3.7 3.3]
[3.2 3.2 3.1 2.3 2.8 2.8 3.3 2.4 2.9 2.7 2.  3.  2.2 2.9 2.9 3.1 3.  2.7
 2.2 2.5 3.2 2.8 2.5 2.8 2.9 3.  2.8 3.  2.9 2.6 2.4 2.4 2.7 2.7 3.  3.4
 3.1 2.3 3.  2.5 2.6 3.  2.6 2.3 2.7 3.  2.9 2.9 2.5 2.8]
[3.3 2.7 3.  2.9 3.  3.  2.5 2.9 2.5 3.6 3.2 2.7 3.  2.5 2.8 3.2 3.  3.8
 2.6 2.2 3.2 2.8 2.8 2.7 3.3 3.2 2.8 3.  2.8 3.  2.8 3.8 2.8 2.8 2.6 3.
 3.4 3.1 3.  3.1 3.1 3.1 2.7 3.2 3.3 3.  2.5 3.  3.4 3. ]


In [9]:
# [3] 품종별 각 변수의 평균 확인
print(gA.mean(), round(gB.mean(), 3), gC.mean())

3.428 2.77 2.974


In [10]:
# [4] 일원분산분석
# 1) model = OLS.from_formula()
# 2) anova_lm(model)
from statsmodels.api import OLS
from statsmodels.stats.anova import anova_lm

formula = 'sepal_width ~ C(species)'
model = OLS.from_formula(formula, data=iris).fit()
result = anova_lm(model)
print(result)

               df     sum_sq   mean_sq         F        PR(>F)
C(species)    2.0  11.344933  5.672467  49.16004  4.492017e-17
Residual    147.0  16.962000  0.115388       NaN           NaN


In [19]:
# [5] 오차제곱합, SSE(Sum of Squares Error)
SSE = result.loc['Residual', 'sum_sq']
print(round(result, 6))

# [6] 회귀제곱합, SSR(Sum of Squares of Regression)
SSR = result.loc['C(species)', 'sum_sq']
print(round(SSR, 6))

# [7] 결정계수
R2 = SSR / (SSR + SSE)
# R2 = SSR / result['sum_sq'].mean()
print(round(R2, 3))

# [8] 평균제곱오차(MSE, Mean Squared Error)
MSE = result.loc['Residual', 'mean_sq']
print(round(MSE, 6))

# [9] 평균제곱회귀(MSR, Mena Squared Regression)
MSR = result.loc['C(species)', 'mean_sq']
print(round(MSR, 6))

# [10] F-통계량
F = result.loc['C(species)', 'F']
print(round(F, 6))

# [11] F-통계량 - pvalue
pvalue = result.loc['C(species)', 'PR(>F)']
print(round(pvalue, 6))

               df     sum_sq   mean_sq         F  PR(>F)
C(species)    2.0  11.344933  5.672467  49.16004     0.0
Residual    147.0  16.962000  0.115388       NaN     NaN
11.344933
0.401
0.115388
5.672467
49.16004
0.0


In [20]:
# [5] 사후검정
from statsmodels.stats.multicomp import pairwise_tukeyhsd
result = pairwise_tukeyhsd(iris['sepal_width'], iris['species'], alpha=0.5)
print(result)

    Multiple Comparison of Means - Tukey HSD, FWER=0.50     
  group1     group2   meandiff p-adj   lower   upper  reject
------------------------------------------------------------
    setosa versicolor   -0.658    0.0 -0.7345 -0.5815   True
    setosa  virginica   -0.454    0.0 -0.5305 -0.3775   True
versicolor  virginica    0.204 0.0088  0.1275  0.2805   True
------------------------------------------------------------
